#  Figure 3

# ECONOMIC ANALYSIS

## FAMILY INCOME AND POVERTY BY RACE/ETHNICITY


In [1]:

from __future__ import division
import numpy as np
import scipy as sc
from itertools import product
import time
import matplotlib.pyplot as plt
import PIL
from numpy import log10
import random
from math import factorial
from scipy.stats import linregress, gaussian_kde, skew
from scipy import stats
from scipy.spatial import distance
import warnings
import pandas as pd
import re
import os
import math
from collections import Counter
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.stats.outliers_influence import summary_table
import statsmodels.api as sm
from scipy.optimize import linear_sum_assignment

warnings.filterwarnings('ignore')

%config InlineBackend.figure_formats = ['svg']
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

pd.set_option('display.max_columns', None)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
def count_pts_within_radius(x, y, radius, scale=0):
    """Count the number of points within a fixed radius in 2D space"""
    
    raw_data = np.array([x, y])
    x = np.array(x)
    y = np.array(y)
    raw_data = raw_data.transpose()
    
    # Get unique data points by adding each pair of points to a set
    unique_points = set()
    for xval, yval in raw_data:
        unique_points.add((xval, yval))
    
    count_data = []
    for a, b in unique_points:
        if scale == 'sqrt':
            num_neighbors = len(x[((sqrt(x) - sqrt(a)) ** 2 +
                                   (sqrt(y) - sqrt(b)) ** 2) <= sqrt(radius) ** 2])
        else:        
            num_neighbors = len(x[((x - a) ** 2 + (y - b) ** 2) <= radius ** 2])
        count_data.append((a, b, num_neighbors))
    return count_data



def plot_color_by_pt_dens(x, y, radius, loglog=0, scale=0, plot_obj=None, point_size=10):
    
    """Plot bivariate relationships with large n using color for point density

    Inputs:
    x & y -- variables to be plotted
    radius -- the linear distance within which to count points as neighbors
    scale -- a flag to indicate the use of a scale plot (scale = 1)

    The color of each point in the plot is determined by the logarithm (base 10)
    of the number of points that occur with a given radius of the focal point,
    with hotter colors indicating more points. The number of neighboring points
    is determined in linear space regardless of whether a scale plot is
    presented.
    """
    
    plot_data = count_pts_within_radius(x, y, radius, scale)
    sorted_plot_data = np.array(sorted(plot_data, key=lambda point: point[2]))

    if plot_obj == None:
        plot_obj = plt.axes()
        
    plot_obj.scatter(sorted_plot_data[:, 0],
            sorted_plot_data[:, 1],
            facecolors='none',
            s = point_size, 
            edgecolors='0.1', 
            linewidths=1., 
            #cmap='Greys'
            )
    
    # plot points
    c = np.array(sorted_plot_data[:, 2])**0.25
    c = np.max(c) - c
    plot_obj.scatter(sorted_plot_data[:, 0],
                    sorted_plot_data[:, 1],
                    c = c,
                    s = point_size, 
                    edgecolors='k', 
                    linewidths=0.0, 
                    cmap='Greys_r',
                    #alpha = 0.5,
                    )
        
    return plot_obj


In [3]:
def DS(p):
    p_bins = len(p)
    p_obs = sum(p)
    
    z_p = (p_bins + 1)/p_bins
    p = [sum(p[:ii+1])**(z_p) for ii in range(len(p))]
    Sp = np.sum(np.array(p)/(p_obs**z_p)) - 1
    ds = Sp/(p_bins - 1)
    return ds


## Family income data

In [4]:
datasets = []
for name in os.listdir('data/economic/USCB/FAMILY_INCOME_IN_THE_PAST_12_MONTHS/data/'):
    datasets.append(name)

main_df = pd.DataFrame(columns=['Year'])

yrs = ['5Y2010', '5Y2011', '5Y2012', '5Y2013', '5Y2014', 
       '5Y2015', '5Y2016', '5Y2017', '5Y2018', '5Y2019', '5Y2020', 
       '5Y2021', '5Y2022']

# Iterate over each year to concatenate files and add a 'year' column

for yr in yrs:
    tdf = pd.DataFrame(columns=['Year'])

    files = []
    for d in datasets:
        if yr in d:
            files.append(d)

    for file in files:
        # Read CSV and specify the header row
        df = pd.read_csv('data/economic/USCB/FAMILY_INCOME_IN_THE_PAST_12_MONTHS/data/'+file, header=0)
        
        # Reset the index of the DataFrame
        df = df.reset_index(drop=True)

        lab = file
        lab = lab.replace('-Data.csv', '')
        lab = lab + '-Table-Notes.txt'

        path = 'data/economic/USCB/FAMILY_INCOME_IN_THE_PAST_12_MONTHS/notes/'+lab
        # Open the file in read mode
        with open(path, "r") as File:
            # Read the first six lines
            first_six_lines = [File.readline() for _ in range(6)]

        # Print the first six lines
        lab1 = first_six_lines[-1]
        lab1 = lab1.upper()

        R_E = ['TWO OR MORE RACES HOUSEHOLDER',
            'BLACK OR AFRICAN AMERICAN ALONE HOUSEHOLDER',
            'WHITE ALONE HOUSEHOLDER',
            'ASIAN ALONE HOUSEHOLDER',
            'HISPANIC OR LATINO HOUSEHOLDER',
            'WHITE ALONE, NOT HISPANIC OR LATINO HOUSEHOLDER',
            'NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE HOUSEHOLDER',
            'AMERICAN INDIAN AND ALASKA NATIVE ALONE HOUSEHOLDER',
           ]

        lab2 = str(lab1)
        for r_e in R_E:
            if r_e in lab1:
                lab2 = r_e

        if lab2 == lab1:
            lab2 = 'ALL'

        # Set new headers, drop redundant rows, and reset index
        new_headers = df.iloc[0]
        df.columns = new_headers
        df = df.drop(0)
        df = df.reset_index(drop=True)

        df = df.drop(columns=['Geography', 'Geographic Area Name'])
        df = df.dropna(axis=1, how='all')

        df.dropna(how='all', axis=1, inplace=True)
        cols_to_drop = df.filter(regex='Margin of Error').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='Median').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='MEDIAN').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='Mean').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='MEAN').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='IMPUTED').columns
        df = df.drop(columns=cols_to_drop)
        
        df.columns = df.columns.str.replace('Total:', 'Total')
        df.columns = df.columns.str.replace("Estimate!!", "")
        df.columns = df.columns.str.replace("Total!!", "")
        
        df.columns = [col + ' - ' + lab2 for col in df.columns]
        df['Year'] = yr[2:]
        
        df.columns = df.columns.str.replace(" HOUSEHOLDER", "")
        df.columns = df.columns.str.replace(" ALONE", "")
        
        # Merge files for same year
        tdf = tdf.merge(df, how='outer', on='Year')
        tdf.reset_index(drop=True, inplace=True)
        
    # concat different years
    try:
        main_df = pd.concat([main_df, tdf])
        #main_df = main_df.sort_index(axis=1)
    except:
        pass
        
print(main_df.shape)
main_df.head(main_df.shape[0])

(13, 154)


,Year,Total - WHITE,"Less than $10,000 - WHITE","$10,000 to $14,999 - WHITE","$15,000 to $19,999 - WHITE","$20,000 to $24,999 - WHITE","$25,000 to $29,999 - WHITE","$30,000 to $34,999 - WHITE","$35,000 to $39,999 - WHITE","$40,000 to $44,999 - WHITE","$45,000 to $49,999 - WHITE","$50,000 to $59,999 - WHITE","$60,000 to $74,999 - WHITE","$75,000 to $99,999 - WHITE","$100,000 to $124,999 - WHITE","$125,000 to $149,999 - WHITE","$150,000 to $199,999 - WHITE","$200,000 or more - WHITE",Total - TWO OR MORE RACES,"Less than $10,000 - TWO OR MORE RACES","$10,000 to $14,999 - TWO OR MORE RACES","$15,000 to $19,999 - TWO OR MORE RACES","$20,000 to $24,999 - TWO OR MORE RACES","$25,000 to $29,999 - TWO OR MORE RACES","$30,000 to $34,999 - TWO OR MORE RACES","$35,000 to $39,999 - TWO OR MORE RACES","$40,000 to $44,999 - TWO OR MORE RACES","$45,000 to $49,999 - TWO OR MORE RACES","$50,000 to $59,999 - TWO OR MORE RACES","$60,000 to $74,999 - TWO OR MORE RACES","$75,000 to $99,999 - TWO OR MORE RACES","$100,000 to $124,999 - TWO OR MORE RACES","$125,000 to $149,999 - TWO OR MORE RACES","$150,000 to $199,999 - TWO OR MORE RACES","$200,000 or more - TWO OR MORE RACES",Total - HISPANIC OR LATINO,"Less than $10,000 - HISPANIC OR LATINO","$10,000 to $14,999 - HISPANIC OR LATINO","$15,000 to $19,999 - HISPANIC OR LATINO","$20,000 to $24,999 - HISPANIC OR LATINO","$25,000 to $29,999 - HISPANIC OR LATINO","$30,000 to $34,999 - HISPANIC OR LATINO","$35,000 to $39,999 - HISPANIC OR LATINO","$40,000 to $44,999 - HISPANIC OR LATINO","$45,000 to $49,999 - HISPANIC OR LATINO","$50,000 to $59,999 - HISPANIC OR LATINO","$60,000 to $74,999 - HISPANIC OR LATINO","$75,000 to $99,999 - HISPANIC OR LATINO","$100,000 to $124,999 - HISPANIC OR LATINO","$125,000 to $149,999 - HISPANIC OR LATINO","$150,000 to $199,999 - HISPANIC OR LATINO","$200,000 or more - HISPANIC OR LATINO","Total - WHITE, NOT HISPANIC OR LATINO","Less than $10,000 - WHITE, NOT HISPANIC OR LATINO","$10,000 to $14,999 - WHITE, NOT HISPANIC OR LATINO","$15,000 to $19,999 - WHITE, NOT HISPANIC OR LATINO","$20,000 to $24,999 - WHITE, NOT HISPANIC OR LATINO","$25,000 to $29,999 - WHITE, NOT HISPANIC OR LATINO","$30,000 to $34,999 - WHITE, NOT HISPANIC OR LATINO","$35,000 to $39,999 - WHITE, NOT HISPANIC OR LATINO","$40,000 to $44,999 - WHITE, NOT HISPANIC OR LATINO","$45,000 to $49,999 - WHITE, NOT HISPANIC OR LATINO","$50,000 to $59,999 - WHITE, NOT HISPANIC OR LATINO","$60,000 to $74,999 - WHITE, NOT HISPANIC OR LATINO","$75,000 to $99,999 - WHITE, NOT HISPANIC OR LATINO","$100,000 to $124,999 - WHITE, NOT HISPANIC OR LATINO","$125,000 to $149,999 - WHITE, NOT HISPANIC OR LATINO","$150,000 to $199,999 - WHITE, NOT HISPANIC OR LATINO","$200,000 or more - WHITE, NOT HISPANIC OR LATINO",Total - AMERICAN INDIAN AND ALASKA NATIVE,"Less than $10,000 - AMERICAN INDIAN AND ALASKA NATIVE","$10,000 to $14,999 - AMERICAN INDIAN AND ALASKA NATIVE","$15,000 to $19,999 - AMERICAN INDIAN AND ALASKA NATIVE","$20,000 to $24,999 - AMERICAN INDIAN AND ALASKA NATIVE","$25,000 to $29,999 - AMERICAN INDIAN AND ALASKA NATIVE","$30,000 to $34,999 - AMERICAN INDIAN AND ALASKA NATIVE","$35,000 to $39,999 - AMERICAN INDIAN AND ALASKA NATIVE","$40,000 to $44,999 - AMERICAN INDIAN AND ALASKA NATIVE","$45,000 to $49,999 - AMERICAN INDIAN AND ALASKA NATIVE","$50,000 to $59,999 - AMERICAN INDIAN AND ALASKA NATIVE","$60,000 to $74,999 - AMERICAN INDIAN AND ALASKA NATIVE","$75,000 to $99,999 - AMERICAN INDIAN AND ALASKA NATIVE","$100,000 to $124,999 - AMERICAN INDIAN AND ALASKA NATIVE","$125,000 to $149,999 - AMERICAN INDIAN AND ALASKA NATIVE","$150,000 to $199,999 - AMERICAN INDIAN AND ALASKA NATIVE","$200,000 or more - AMERICAN INDIAN AND ALASKA NATIVE",Total - BLACK OR AFRICAN AMERICAN,"Less than $10,000 - BLACK OR AFRICAN AMERICAN","$10,000 to $14,999 - BLACK OR AFRICAN AMERICAN","$15,000 to $19,999 - BLACK OR AFRICAN AMERICAN","$20,000 to $24,999 - BLACK OR AFRICAN AMERICAN","$25,000 to $29,999 - BLACK OR AFRICAN AMERICAN",

In [5]:
suf_ls = ['WHITE',
          'HISPANIC OR LATINO',
          'WHITE, NOT HISPANIC OR LATINO',
          'BLACK OR AFRICAN AMERICAN',
          'ASIAN',
          'ALL',
          'NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER',
          'AMERICAN INDIAN AND ALASKA NATIVE',
         ]

for suf in suf_ls:
    # Define columns to convert to float

    print(suf)
    cols = ['Less than $10,000 - '+suf, 
        '$10,000 to $14,999 - '+suf, 
        '$15,000 to $19,999 - '+suf, 
        '$20,000 to $24,999 - '+suf, 
        '$25,000 to $29,999 - '+suf, 
        '$30,000 to $34,999 - '+suf, 
        '$35,000 to $39,999 - '+suf, 
        '$40,000 to $44,999 - '+suf, 
        '$45,000 to $49,999 - '+suf, 
        '$50,000 to $59,999 - '+suf, 
        '$60,000 to $74,999 - '+suf, 
        '$75,000 to $99,999 - '+suf, 
        '$100,000 to $124,999 - '+suf, 
        '$125,000 to $149,999 - '+suf, 
        '$150,000 to $199,999 - '+suf, 
        '$200,000 or more - '+suf,
       ]

    main_df[cols] = main_df[cols].astype(float)
    main_df['DOW'] = main_df.apply(lambda row: row[cols].tolist(), axis=1)

    ds_ls = []
    for dow in main_df['DOW'].tolist():
        ds = DS(dow)
        ds_ls.append(ds)

    main_df[suf+'_DS'] = ds_ls
    main_df.drop(labels=['DOW'], axis=1, inplace=True)
    
    main_df[suf + ' < $25K'] = 100 * (main_df[cols[0]] + main_df[cols[1]] + main_df[cols[2]] + main_df[cols[3]]) / main_df['Total - ' + suf].astype(float)
    main_df[suf + ' ≥ $200K'] = 100 * (main_df[cols[-1]]) / main_df['Total - ' + suf].astype(float)

    ##
    ## Get DS for lower
    ##
    
    cols = ['Less than $10,000 - '+suf, 
        '$10,000 to $14,999 - '+suf, 
        '$15,000 to $19,999 - '+suf, 
        '$20,000 to $24,999 - '+suf, 
        ]

    main_df['DOW'] = main_df.apply(lambda row: row[cols].tolist(), axis=1)

    ds_ls = []
    for dow in main_df['DOW'].tolist():
        ds = DS(dow)
        ds_ls.append(ds)

    main_df[suf+'_DS_lower'] = ds_ls
    main_df.drop(labels=['DOW'], axis=1, inplace=True)
    
    ##
    ## Get DS for middle
    ##
    
    cols = ['$25,000 to $29,999 - '+suf, 
        '$30,000 to $34,999 - '+suf, 
        '$35,000 to $39,999 - '+suf, 
        '$40,000 to $44,999 - '+suf,
        '$45,000 to $49,999 - '+suf, 
        '$50,000 to $59,999 - '+suf, 
        '$60,000 to $74,999 - '+suf, 
        '$75,000 to $99,999 - '+suf, 
       ]

    main_df['DOW'] = main_df.apply(lambda row: row[cols].tolist(), axis=1)

    ds_ls = []
    for dow in main_df['DOW'].tolist():
        ds = DS(dow)
        ds_ls.append(ds)

    main_df[suf+'_DS_middle'] = ds_ls
    main_df.drop(labels=['DOW'], axis=1, inplace=True)
    
    ##
    ## Get DS for upper
    ##
    
    cols = [
        '$100,000 to $124,999 - '+suf, 
        '$125,000 to $149,999 - '+suf, 
        '$150,000 to $199,999 - '+suf, 
        '$200,000 or more - '+suf,
       ]
    
    main_df['DOW'] = main_df.apply(lambda row: row[cols].tolist(), axis=1)

    ds_ls = []
    for dow in main_df['DOW'].tolist():
        ds = DS(dow)
        ds_ls.append(ds)

    main_df[suf+'_DS_upper'] = ds_ls
    main_df.drop(labels=['DOW'], axis=1, inplace=True)

main_df.head(main_df.shape[0])

WHITE
HISPANIC OR LATINO
WHITE, NOT HISPANIC OR LATINO
BLACK OR AFRICAN AMERICAN
ASIAN
ALL
NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER
AMERICAN INDIAN AND ALASKA NATIVE


,Year,Total - WHITE,"Less than $10,000 - WHITE","$10,000 to $14,999 - WHITE","$15,000 to $19,999 - WHITE","$20,000 to $24,999 - WHITE","$25,000 to $29,999 - WHITE","$30,000 to $34,999 - WHITE","$35,000 to $39,999 - WHITE","$40,000 to $44,999 - WHITE","$45,000 to $49,999 - WHITE","$50,000 to $59,999 - WHITE","$60,000 to $74,999 - WHITE","$75,000 to $99,999 - WHITE","$100,000 to $124,999 - WHITE","$125,000 to $149,999 - WHITE","$150,000 to $199,999 - WHITE","$200,000 or more - WHITE",Total - TWO OR MORE RACES,"Less than $10,000 - TWO OR MORE RACES","$10,000 to $14,999 - TWO OR MORE RACES","$15,000 to $19,999 - TWO OR MORE RACES","$20,000 to $24,999 - TWO OR MORE RACES","$25,000 to $29,999 - TWO OR MORE RACES","$30,000 to $34,999 - TWO OR MORE RACES","$35,000 to $39,999 - TWO OR MORE RACES","$40,000 to $44,999 - TWO OR MORE RACES","$45,000 to $49,999 - TWO OR MORE RACES","$50,000 to $59,999 - TWO OR MORE RACES","$60,000 to $74,999 - TWO OR MORE RACES","$75,000 to $99,999 - TWO OR MORE RACES","$100,000 to $124,999 - TWO OR MORE RACES","$125,000 to $149,999 - TWO OR MORE RACES","$150,000 to $199,999 - TWO OR MORE RACES","$200,000 or more - TWO OR MORE RACES",Total - HISPANIC OR LATINO,"Less than $10,000 - HISPANIC OR LATINO","$10,000 to $14,999 - HISPANIC OR LATINO","$15,000 to $19,999 - HISPANIC OR LATINO","$20,000 to $24,999 - HISPANIC OR LATINO","$25,000 to $29,999 - HISPANIC OR LATINO","$30,000 to $34,999 - HISPANIC OR LATINO","$35,000 to $39,999 - HISPANIC OR LATINO","$40,000 to $44,999 - HISPANIC OR LATINO","$45,000 to $49,999 - HISPANIC OR LATINO","$50,000 to $59,999 - HISPANIC OR LATINO","$60,000 to $74,999 - HISPANIC OR LATINO","$75,000 to $99,999 - HISPANIC OR LATINO","$100,000 to $124,999 - HISPANIC OR LATINO","$125,000 to $149,999 - HISPANIC OR LATINO","$150,000 to $199,999 - HISPANIC OR LATINO","$200,000 or more - HISPANIC OR LATINO","Total - WHITE, NOT HISPANIC OR LATINO","Less than $10,000 - WHITE, NOT HISPANIC OR LATINO","$10,000 to $14,999 - WHITE, NOT HISPANIC OR LATINO","$15,000 to $19,999 - WHITE, NOT HISPANIC OR LATINO","$20,000 to $24,999 - WHITE, NOT HISPANIC OR LATINO","$25,000 to $29,999 - WHITE, NOT HISPANIC OR LATINO","$30,000 to $34,999 - WHITE, NOT HISPANIC OR LATINO","$35,000 to $39,999 - WHITE, NOT HISPANIC OR LATINO","$40,000 to $44,999 - WHITE, NOT HISPANIC OR LATINO","$45,000 to $49,999 - WHITE, NOT HISPANIC OR LATINO","$50,000 to $59,999 - WHITE, NOT HISPANIC OR LATINO","$60,000 to $74,999 - WHITE, NOT HISPANIC OR LATINO","$75,000 to $99,999 - WHITE, NOT HISPANIC OR LATINO","$100,000 to $124,999 - WHITE, NOT HISPANIC OR LATINO","$125,000 to $149,999 - WHITE, NOT HISPANIC OR LATINO","$150,000 to $199,999 - WHITE, NOT HISPANIC OR LATINO","$200,000 or more - WHITE, NOT HISPANIC OR LATINO",Total - AMERICAN INDIAN AND ALASKA NATIVE,"Less than $10,000 - AMERICAN INDIAN AND ALASKA NATIVE","$10,000 to $14,999 - AMERICAN INDIAN AND ALASKA NATIVE","$15,000 to $19,999 - AMERICAN INDIAN AND ALASKA NATIVE","$20,000 to $24,999 - AMERICAN INDIAN AND ALASKA NATIVE","$25,000 to $29,999 - AMERICAN INDIAN AND ALASKA NATIVE","$30,000 to $34,999 - AMERICAN INDIAN AND ALASKA NATIVE","$35,000 to $39,999 - AMERICAN INDIAN AND ALASKA NATIVE","$40,000 to $44,999 - AMERICAN INDIAN AND ALASKA NATIVE","$45,000 to $49,999 - AMERICAN INDIAN AND ALASKA NATIVE","$50,000 to $59,999 - AMERICAN INDIAN AND ALASKA NATIVE","$60,000 to $74,999 - AMERICAN INDIAN AND ALASKA NATIVE","$75,000 to $99,999 - AMERICAN INDIAN AND ALASKA NATIVE","$100,000 to $124,999 - AMERICAN INDIAN AND ALASKA NATIVE","$125,000 to $149,999 - AMERICAN INDIAN AND ALASKA NATIVE","$150,000 to $199,999 - AMERICAN INDIAN AND ALASKA NATIVE","$200,000 or more - AMERICAN INDIAN AND ALASKA NATIVE",Total - BLACK OR AFRICAN AMERICAN,"Less than $10,000 - BLACK OR AFRICAN AMERICAN","$10,000 to $14,999 - BLACK OR AFRICAN AMERICAN","$15,000 to $19,999 - BLACK OR AFRICAN AMERICAN","$20,000 to $24,999 - BLACK OR AFRICAN AMERICAN","$25,000 to $29,999 - BLACK OR AFRICAN AMERICAN",

In [6]:
datasets = []
for name in os.listdir('data/economic/USCB/POVERTY/data/'):
    datasets.append(name)

df2 = pd.DataFrame(columns=['Year'])

yrs = ['1Y2010', '1Y2011', '1Y2012', '1Y2013', 
       '1Y2014', '1Y2015', '1Y2016', 
       '5Y2017', '5Y2018', '5Y2019', '5Y2020', 
       '5Y2021', '5Y2022',
      ]

# Iterate over each year to concatenate files and add a 'year' column

for yr in yrs:
    tdf = pd.DataFrame(columns=['Year'])

    files = []
    for d in datasets:
        if yr in d:
            files.append(d)

    for file in files:
        # Read CSV file and specify header row
        df = pd.read_csv('data/economic/USCB/POVERTY/data/'+file, header=0)
        
        # Reset index of DataFrame
        df = df.reset_index(drop=True)

        # Set new headers, drop redundant rows, and reset index
        new_headers = df.iloc[0]
        df.columns = new_headers
        df = df.drop(0)
        df = df.reset_index(drop=True)
        
        df.columns = df.columns.str.replace('One race!!White', 'White')
        df.columns = df.columns.str.replace('One race!!Black or African American', 'Black or African American')
        df.columns = df.columns.str.replace('One race!!American Indian and Alaska Native', 'American Indian and Alaska Native')
        df.columns = df.columns.str.replace('One race!!Asian', 'Asian')
        df.columns = df.columns.str.replace('One race!!Native Hawaiian and Other Pacific Islander', 'Native Hawaiian and Other Pacific Islander')
        
        df.columns = df.columns.str.replace('White alone', 'White')
        df.columns = df.columns.str.replace('Black or African American alone', 'Black or African American')
        df.columns = df.columns.str.replace('American Indian and Alaska Native alone', 'American Indian and Alaska Native')
        df.columns = df.columns.str.replace('Asian alone', 'Asian')
        df.columns = df.columns.str.replace('Native Hawaiian and Other Pacific Islander alone', 'Native Hawaiian and Other Pacific Islander')
        
        df.columns = df.columns.str.replace("\(of any race\)", "")
        
        df.columns = df.columns.str.replace("Estimate!!Total!!", "Total!!Estimate!!")
        df.columns = df.columns.str.replace("Estimate!!Below poverty level!!", "Below poverty level!!Estimate!!")
        
        df.columns = df.columns.str.replace("Population for whom poverty status is determined!!", "")
        
        df.columns = df.columns.str.replace("RACE AND HISPANIC OR LATINO ORIGIN!!", "")
        
        
        df = df.drop(columns=['Geography', 'Geographic Area Name'])
        cols_to_drop = df.filter(regex='Margin of Error').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='Median').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='MEDIAN').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='Mean').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='MEAN').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='IMPUTED').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='years').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='Female').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='Male').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='housing').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='ork').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='raduate').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='egree').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='ndividuals').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='NDIVIDUALS').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='mploy').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='other').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='or more race').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='One race').columns
        df = df.drop(columns=cols_to_drop)
        cols_to_drop = df.filter(regex='Percent').columns
        df = df.drop(columns=cols_to_drop)
        
        df.dropna(how='all', axis=1, inplace=True)
        #df.columns = df.columns.str.replace('Total:', 'Total')

        df['Year'] = yr[2:]

        # Merge files for same year
        tdf = tdf.merge(df, how='outer', on='Year')
        tdf.reset_index(drop=True, inplace=True)
        
    # Now concat different years
    try:
        df2 = pd.concat([df2, tdf])
        #main_df = main_df.sort_index(axis=1)
    except:
        pass


df.reset_index(drop=True, inplace=True)
print(df2.shape)
df2.head(df2.shape[0])


suf1_ls = ['White',
           'Hispanic or Latino origin ',
           'White, not Hispanic or Latino',
           'Black or African American',
           'Asian',
           'Population for whom poverty status is determined',
           'Native Hawaiian and Other Pacific Islander',
           'American Indian and Alaska Native',
          ]

suf2_ls = ['WHITE',
          'HISPANIC OR LATINO',
          'WHITE, NOT HISPANIC OR LATINO',
          'BLACK OR AFRICAN AMERICAN',
          'ASIAN',
          'ALL',
          'NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER',
          'AMERICAN INDIAN AND ALASKA NATIVE',
         ]

        
for i, suf in enumerate(suf1_ls):
    
    pvr = 100 * df2['Below poverty level!!Estimate!!' + suf].astype(float) / df2['Total!!Estimate!!' + suf].astype(float)
    
    main_df[suf2_ls[i] + ' - % Poverty'] = pvr
    
main_df.head(main_df.shape[0])     

(13, 17)


,Year,Total - WHITE,"Less than $10,000 - WHITE","$10,000 to $14,999 - WHITE","$15,000 to $19,999 - WHITE","$20,000 to $24,999 - WHITE","$25,000 to $29,999 - WHITE","$30,000 to $34,999 - WHITE","$35,000 to $39,999 - WHITE","$40,000 to $44,999 - WHITE","$45,000 to $49,999 - WHITE","$50,000 to $59,999 - WHITE","$60,000 to $74,999 - WHITE","$75,000 to $99,999 - WHITE","$100,000 to $124,999 - WHITE","$125,000 to $149,999 - WHITE","$150,000 to $199,999 - WHITE","$200,000 or more - WHITE",Total - TWO OR MORE RACES,"Less than $10,000 - TWO OR MORE RACES","$10,000 to $14,999 - TWO OR MORE RACES","$15,000 to $19,999 - TWO OR MORE RACES","$20,000 to $24,999 - TWO OR MORE RACES","$25,000 to $29,999 - TWO OR MORE RACES","$30,000 to $34,999 - TWO OR MORE RACES","$35,000 to $39,999 - TWO OR MORE RACES","$40,000 to $44,999 - TWO OR MORE RACES","$45,000 to $49,999 - TWO OR MORE RACES","$50,000 to $59,999 - TWO OR MORE RACES","$60,000 to $74,999 - TWO OR MORE RACES","$75,000 to $99,999 - TWO OR MORE RACES","$100,000 to $124,999 - TWO OR MORE RACES","$125,000 to $149,999 - TWO OR MORE RACES","$150,000 to $199,999 - TWO OR MORE RACES","$200,000 or more - TWO OR MORE RACES",Total - HISPANIC OR LATINO,"Less than $10,000 - HISPANIC OR LATINO","$10,000 to $14,999 - HISPANIC OR LATINO","$15,000 to $19,999 - HISPANIC OR LATINO","$20,000 to $24,999 - HISPANIC OR LATINO","$25,000 to $29,999 - HISPANIC OR LATINO","$30,000 to $34,999 - HISPANIC OR LATINO","$35,000 to $39,999 - HISPANIC OR LATINO","$40,000 to $44,999 - HISPANIC OR LATINO","$45,000 to $49,999 - HISPANIC OR LATINO","$50,000 to $59,999 - HISPANIC OR LATINO","$60,000 to $74,999 - HISPANIC OR LATINO","$75,000 to $99,999 - HISPANIC OR LATINO","$100,000 to $124,999 - HISPANIC OR LATINO","$125,000 to $149,999 - HISPANIC OR LATINO","$150,000 to $199,999 - HISPANIC OR LATINO","$200,000 or more - HISPANIC OR LATINO","Total - WHITE, NOT HISPANIC OR LATINO","Less than $10,000 - WHITE, NOT HISPANIC OR LATINO","$10,000 to $14,999 - WHITE, NOT HISPANIC OR LATINO","$15,000 to $19,999 - WHITE, NOT HISPANIC OR LATINO","$20,000 to $24,999 - WHITE, NOT HISPANIC OR LATINO","$25,000 to $29,999 - WHITE, NOT HISPANIC OR LATINO","$30,000 to $34,999 - WHITE, NOT HISPANIC OR LATINO","$35,000 to $39,999 - WHITE, NOT HISPANIC OR LATINO","$40,000 to $44,999 - WHITE, NOT HISPANIC OR LATINO","$45,000 to $49,999 - WHITE, NOT HISPANIC OR LATINO","$50,000 to $59,999 - WHITE, NOT HISPANIC OR LATINO","$60,000 to $74,999 - WHITE, NOT HISPANIC OR LATINO","$75,000 to $99,999 - WHITE, NOT HISPANIC OR LATINO","$100,000 to $124,999 - WHITE, NOT HISPANIC OR LATINO","$125,000 to $149,999 - WHITE, NOT HISPANIC OR LATINO","$150,000 to $199,999 - WHITE, NOT HISPANIC OR LATINO","$200,000 or more - WHITE, NOT HISPANIC OR LATINO",Total - AMERICAN INDIAN AND ALASKA NATIVE,"Less than $10,000 - AMERICAN INDIAN AND ALASKA NATIVE","$10,000 to $14,999 - AMERICAN INDIAN AND ALASKA NATIVE","$15,000 to $19,999 - AMERICAN INDIAN AND ALASKA NATIVE","$20,000 to $24,999 - AMERICAN INDIAN AND ALASKA NATIVE","$25,000 to $29,999 - AMERICAN INDIAN AND ALASKA NATIVE","$30,000 to $34,999 - AMERICAN INDIAN AND ALASKA NATIVE","$35,000 to $39,999 - AMERICAN INDIAN AND ALASKA NATIVE","$40,000 to $44,999 - AMERICAN INDIAN AND ALASKA NATIVE","$45,000 to $49,999 - AMERICAN INDIAN AND ALASKA NATIVE","$50,000 to $59,999 - AMERICAN INDIAN AND ALASKA NATIVE","$60,000 to $74,999 - AMERICAN INDIAN AND ALASKA NATIVE","$75,000 to $99,999 - AMERICAN INDIAN AND ALASKA NATIVE","$100,000 to $124,999 - AMERICAN INDIAN AND ALASKA NATIVE","$125,000 to $149,999 - AMERICAN INDIAN AND ALASKA NATIVE","$150,000 to $199,999 - AMERICAN INDIAN AND ALASKA NATIVE","$200,000 or more - AMERICAN INDIAN AND ALASKA NATIVE",Total - BLACK OR AFRICAN AMERICAN,"Less than $10,000 - BLACK OR AFRICAN AMERICAN","$10,000 to $14,999 - BLACK OR AFRICAN AMERICAN","$15,000 to $19,999 - BLACK OR AFRICAN AMERICAN","$20,000 to $24,999 - BLACK OR AFRICAN AMERICAN","$25,000 to $29,999 - BLACK OR AFRICAN AMERICAN",

In [7]:
race_colors = {
    'BLACK OR AFRICAN AMERICAN': '0.8',
    'AMERICAN INDIAN AND ALASKA NATIVE': '0.8',
    'HISPANIC OR LATINO': '0.8',
    'ALL': 'k',
    'WHITE, NOT HISPANIC OR LATINO': '0.1',
    'ASIAN': '0.1',
    #'NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER': '0.5',
    
}

race_styles = {
    'BLACK OR AFRICAN AMERICAN': 'solid',
    'AMERICAN INDIAN AND ALASKA NATIVE': 'dashed',
    'HISPANIC OR LATINO': 'dotted',
    'ALL': 'dotted',
    'WHITE, NOT HISPANIC OR LATINO': 'solid',
    'ASIAN': 'dashed',
    #'NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER': '0.5',
}

race_labels = {
    'HISPANIC OR LATINO': 'Hispanic or Latinx',
    'WHITE, NOT HISPANIC OR LATINO': 'White, non-hispanic/latinx',
    'BLACK OR AFRICAN AMERICAN': 'Black or African American',
    'ASIAN': 'Asian',
    'ALL': 'All',
    'NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER': 'Native Hawaiian or Pacific Islander',
    'AMERICAN INDIAN AND ALASKA NATIVE': 'Native American or Alaska Native',
}

# Use the race/ethnicity labels
race_eth = ['AMERICAN INDIAN AND ALASKA NATIVE',
            'BLACK OR AFRICAN AMERICAN',
            'HISPANIC OR LATINO',
            #'ALL',
            'WHITE, NOT HISPANIC OR LATINO',
            'ASIAN',
            #'NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER',
           ]
  
# Use these years
years = ['2010', '', '2012', '', '2014', 
         '', '2016', '', '2018', '', '2020',
         '', '2022',
        ]


# Create figure and axis objects
fig, axs = plt.subplots(1, 3, figsize=(13, 4))

# Plot poverty vs year
c = 0
for race in race_eth:
    axs[c].plot(main_df['Year'], main_df[f'{race} - % Poverty'], 
                label=race_labels[race], 
                color=race_colors[race], 
                linewidth=3, 
                linestyle=race_styles[race],
               )
    
axs[c].set_xlabel('Year', fontsize=14, fontweight='bold')
axs[c].set_ylabel('US Poverty Rate (%)', fontsize=14, fontweight='bold')
axs[c].tick_params(axis='x', rotation=45) 
axs[c].set_xticklabels(years) 

# Plot DS vs year
c = 1
for race in race_eth:
    axs[c].plot(main_df['Year'], main_df[f'{race}_DS'], 
                color=race_colors[race], 
                linewidth=3, 
                linestyle=race_styles[race],
               )
    
axs[c].set_xlabel('Year', fontsize=14, fontweight='bold')
axs[c].set_ylabel('US Poverty (DS)', fontsize=14, fontweight='bold')
axs[c].tick_params(axis='x', rotation=45)
axs[c].set_xticklabels(years) 


# Plot poverty vs year
c = 2
for race in race_eth:
    axs[c].plot(main_df['Year'], main_df[f'{race}_DS_lower'], 
                color=race_colors[race], 
                linewidth=3,
                linestyle=race_styles[race],
               )
axs[c].set_xlabel('Year', fontsize=14, fontweight='bold')
axs[c].set_ylabel('Poverty (DS) of families\nmaking <$25K/year', fontsize=14, fontweight='bold')
axs[c].tick_params(axis='x', rotation=45) 
#axs[c].set_title('Families making <$25K per year', fontsize=14, fontweight='bold')
axs[c].set_xticklabels(years)


fig.patch.set_facecolor('white')
plt.tight_layout()
fig.legend(bbox_to_anchor=(0.055, 1.05, 0.937, .1), 
           loc=10, 
           ncol=3, 
           mode="expand",
           prop={'size':14},
          )

plt.subplots_adjust(wspace=0.4, hspace=0.4)
plt.savefig('Final_Figs/manuscript/Fig3.pdf', bbox_inches='tight', format='pdf', dpi=600)
plt.savefig('Final_Figs/manuscript/Fig3.jpg', bbox_inches='tight', format='jpg', dpi=600)
plt.close()
